In [1]:
using System.IO;

var input = File.ReadAllLines("input.txt");

var q = from y in Enumerable.Range(0, input.Length)
        from x in Enumerable.Range(0, input[y].Length)
        select (x, y, c:input[y][x]);

var grid = q.ToDictionary(x => new Coordinate(x.x,x.y), x => x.c);
var locations  = grid.Where(kv => char.IsDigit(kv.Value)).Select(kv => kv.Key).ToList();
var start = locations.Single(c => grid[c] == '0');
var maxX = grid.Max(kv => kv.Key.x);
var maxY = grid.Max(kv => kv.Key.y);

foreach (var location in locations)
{
    var queue = new Queue<Coordinate>();
    queue.Enqueue(start);
    var remaining = new HashSet<Coordinate>(locations);
    var distances = new Dictionary<int,int>();
    var visited = new HashSet<Coordinate>();

    while (queue.Any() && remaining.Any())
    {
        var current = queue.Dequeue();

        if (visited.Contains(current))
        {
            continue;
        }

        visited.Add(current);

        foreach (var n in current.Neighbours(maxX, maxY))
        {
            if (grid[n] != '#')
                queue.Enqueue(n);
        }

    }

}

*/
start.Display();

readonly record struct Coordinate(int x, int y)
{
    public IEnumerable<Coordinate> Neighbours(int maxX, int maxY)
    {
        if (x > 0) yield return this with { x = x - 1 };
        if (y > 0) yield return this with { y = y - 1 };
        if (x < maxX) yield return this with { x = x + 1 };
        if (y < maxY) yield return this with { y = y + 1 };
    }
}

index,x,y
0,143,1
1,167,9
2,35,11
3,5,19
4,175,21
5,13,31
6,159,31
7,53,37


x,y
35,11
